In [1]:
!pip install --no-deps https://github.com/block-hczhai/block2-preview/releases/download/p0.5.1rc9/block2-0.5.1rc9-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 153.3 MB 13 kB/s 


In [4]:

import numpy as np
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

def perform_dmrg(local_spin, global_spins, topology, n_sites):

    driver = DMRGDriver(scratch='./tmp', symm_type=SymmetryTypes.SU2,
                        n_threads=4)
    driver.initialize_system(n_sites=n_sites, spin=0, heis_twos=local_spin)

    b = driver.expr_builder()
    for (i, j, v) in topology:
        assert i != j
        b.add_term("(T+T)0", [i, j], -np.sqrt(3) / 2 * v)
    heis_mpo = driver.get_mpo(b.finalize(), iprint=0)

    bond_dims = [200] * 5 + [400] * 5 + [800] * 5
    noises = [1E-4] * 5 + [1E-5] * 5 + [0]
    dav_thrds = [1E-7] * 5 + [1E-9] * 5 + [1E-10]

    all_eners = []

    print("\n === S = %d/2 Heisenberg %d-Site Model === \n" %
          (local_spin, n_sites))
    for spin in global_spins:
        target = driver.target
        target.twos = target.twos_low = spin
        ket = driver.get_random_mps(tag='GS', bond_dim=200, nroots=12)
        eners = driver.dmrg(heis_mpo, ket, n_sweeps=20, iprint=0,
                                  bond_dims=bond_dims, noises=noises,
                                  thrds=dav_thrds, dav_max_iter=100, cutoff=0)
        all_eners.append(eners)
        print('2S = %d : E =' % spin, ('%14.8f' * len(eners)) % tuple(eners))

    print("\nRelative energies:\n")
    ener_min = np.min([np.min(x) for x in all_eners])
    for spin, eners in zip(global_spins, all_eners):
        eners = np.array(eners) - ener_min
        print('2S = %d : E =' % spin, ('%14.8f' * len(eners)) % tuple(eners))

In [5]:

n_sites = 4
topology = [
    (1, 2, 1.17),
    (3, 4, 1.17),
    (1, 4, 1.00),
    (2, 3, 1.00),
    (1, 3, 1.00),
    (2, 4, 1.00),
]
topology = [(i - 1, j - 1, k) for i, j, k in topology]

perform_dmrg(local_spin=5, global_spins=[0, 2, 4],
             topology=topology, n_sites=n_sites)

perform_dmrg(local_spin=1, global_spins=[0, 2, 4],
             topology=topology, n_sites=n_sites)


 === S = 5/2 Heisenberg 4-Site Model === 

2S = 0 : E =   -20.47500000  -20.13500000  -19.45500000  -18.43500000  -17.07500000  -15.37500000
2S = 2 : E =   -19.30500000  -19.30500000  -19.13500000  -18.79500000  -18.79500000  -18.45500000  -17.94500000  -17.94500000  -17.43500000  -16.75500000  -16.75500000  -16.07500000
2S = 4 : E =   -17.13500000  -16.96500000  -16.96500000  -16.79500000  -16.79500000  -16.45500000  -16.28500000  -16.28500000  -15.94500000  -15.94500000  -15.43500000  -15.26500000

Relative energies:

2S = 0 : E =     0.00000000    0.34000000    1.02000000    2.04000000    3.40000000    5.10000000
2S = 2 : E =     1.17000000    1.17000000    1.34000000    1.68000000    1.68000000    2.02000000    2.53000000    2.53000000    3.04000000    3.72000000    3.72000000    4.40000000
2S = 4 : E =     3.34000000    3.51000000    3.51000000    3.68000000    3.68000000    4.02000000    4.19000000    4.19000000    4.53000000    4.53000000    5.04000000    5.21000000

 === S = 1

In [6]:

n_sites = 8
topology = [
    (1, 2, 1.00),
    (2, 3, 1.00),
    (3, 4, 1.00),
    (1, 4, 1.00),
    (1, 3, 1.00),
    (2, 4, 1.00),

    (5, 6, 1.00),
    (5, 7, 1.00),
    (6, 8, 1.00),
    (7, 8, 1.00),
    (5, 8, 1.00),
    (6, 7, 1.00),

    (2, 5, 1.00),
    (2, 6, 1.00),
    (2, 7, 1.00),

    (3, 5, 1.00),
    (3, 6, 1.00),
    (3, 7, 1.00),

    (4, 5, 1.00),
    (4, 6, 1.00),
    (4, 7, 1.00),
]
topology = [(i - 1, j - 1, k) for i, j, k in topology]

perform_dmrg(local_spin=5, global_spins=[0, 2, 4, 6, 8],
             topology=topology, n_sites=n_sites)

perform_dmrg(local_spin=1, global_spins=[0, 2, 4, 6, 8],
             topology=topology, n_sites=n_sites)


 === S = 5/2 Heisenberg 8-Site Model === 

2S = 0 : E =   -65.86207906  -60.80762248  -60.80762248  -60.80762248  -60.80762248  -55.73563435  -55.73563435  -55.73563435  -55.73563435  -55.73563435  -55.73563435  -55.73563435
2S = 2 : E =   -64.96068249  -62.38952344  -62.38952344  -62.38952344  -62.38952344  -59.93521509  -59.93521509  -59.93521509  -59.93521509  -57.34360716  -57.34360716  -57.34360716
2S = 4 : E =   -63.15543267  -60.61151216  -60.61151216  -60.61151216  -60.61151216  -58.18592103  -58.18592103  -58.18592103  -58.18592103  -57.92121364  -57.92121364  -57.92121364
2S = 6 : E =   -60.44168361  -57.93654347  -57.93654347  -57.93654347  -57.93654347  -55.55144165  -55.55144165  -55.55144165  -55.55144165  -55.30022075  -55.30022075  -55.30022075
2S = 8 : E =   -56.81307061  -54.35615009  -54.35615009  -54.35615009  -54.35615009  -52.02077188  -52.02077188  -52.02077188  -52.02077188  -51.78619083  -51.78619083  -51.78619083

Relative energies:

2S = 0 : E =     0.000000